<a href="https://colab.research.google.com/github/efforter/Image_Reconstruct_And_Colourful/blob/main/%EC%98%9B%EB%82%A0_%EC%82%AC%EC%A7%84_%EB%B3%B5%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/cszn/BSRGAN.git

Cloning into 'BSRGAN'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 541 (delta 144), reused 103 (delta 101), pack-reused 386
Receiving objects: 100% (541/541), 168.90 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [ ]:
import torch
import cv2
import os
import sys
sys.path.append("/content/BSRGAN")
from utils import utils_image as util
from models.network_rrdbnet import RRDBNet as net
import main_download_pretrained_models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_path = "IRAC_practice_img.jpg"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = net(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, sf=4)
model.load_state_dict(torch.load(os.path.join('/content/BSRGAN/model_zoo', 'BSRGAN.pth')), strict=True)
model = model.to(device)
model.eval()

[3, 3, 64, 23, 32, 4]


RRDBNet(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), str

In [ ]:
with torch.no_grad():
  img = cv2.imread(img_path)

  img_L = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
  img_L = util.uint2tensor4(img_L)
  img_L = img_L.to(device)

  img_E = model(img_L)

  img_E = util.tensor2uint(img_E)
  util.imsave(img_E, os.path.splitext(img_path)[0] + '_result.png')